# Úkol č. 2 - předzpracování dat a binární klasifikace (do 10. listopadu)

  * Cílem thoto úkolu je vyzkoušet si naučit prediktivní model pro binární klasifikaci.
  * Budete se muset vypořádat s příznaky, které jsou různých typů a které bude třeba nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu (preferujeme ale použití cross-validation).
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací/vaidační množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. **První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

In [209]:
### odtud už je to Vaše

import math
import pandas as pd
import numpy as np

In [210]:
# 1] načíst a upravit data 

data = pd.read_csv('data.csv')
# data.info()


# rozlisit nominalni a ordinalni neciselne priznaky a prevest je na ciselne
def edit_oddil(val):
    if type(val) != float:
        return val[:1]
    else: 
        return np.nan

data['oddil'] = data['cabin'].apply(edit_oddil)

n = data['home.dest'].str.rsplit(',',1,expand=True)
data['home.dest'] = n[1]

n = data['name'].str.rsplit(',',1,expand=True)[1].str.split('.',1,expand=True)
data['osloveni'] = n[0]

data = data.drop(columns=['cabin','name','ticket'],axis=1)
# display(data['oddil'].unique())

import string
mappers = {}
mappers['oddil'] = {ch: n for n, ch in enumerate(string.ascii_uppercase)}

# label encoding zpusob predzpracovani
from sklearn.preprocessing import LabelEncoder
def encode_categories(df, mappers, dummies=False):
    le = LabelEncoder()
    for col in df.select_dtypes('object').columns:
        if col not in mappers and df[col].nunique() < 100:
            df[col] = df[col].fillna('NaN')
            df[col] = le.fit_transform(df[col])
            if dummies:
                prefix = 'd_' + col
                df = pd.concat([df.drop(columns=[col]), pd.get_dummies(df[col], prefix=prefix)], axis=1)
        elif col in mappers:
            df[col] = df[col].replace(mappers[col])
        else:
            df[col] = df[col].fillna('NaN')
    return df        

data = encode_categories(data,mappers)
data = data.replace(np.nan,-1)
display(data)


,ID,survived,pclass,sex,age,sibsp,parch,fare,embarked,home.dest,oddil,osloveni
0,0,1,3,0,22.0,0,0,7.7500,3,86,-1.0,9
1,1,0,3,1,-1.0,0,0,8.4583,2,86,-1.0,12
2,2,1,1,0,19.0,1,0,91.0792,0,38,1.0,13
3,3,0,3,1,25.0,0,0,7.2250,0,86,-1.0,12
4,4,0,3,1,-1.0,0,0,7.7500,2,51,-1.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,1,1,0,48.0,1,1,79.2000,0,75,1.0,13
996,996,1,2,0,3.0,1,2,41.5792,0,86,-1.0,9
997,997,1,3,0,-1.0,0,0,7.7500,2,86,-1.0,9
998,998,1,3,1,-1.0,0,0,7.7500,2,86,-1.0,12


In [211]:
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics

In [212]:
# rozdel data na testovaci a trenovaci sadu

ydata = data['survived']
xdata = data.loc[:, data.columns != 'survived']

rd_seed = 333
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.25, random_state=rd_seed)
xtr, xval, ytr, yval = train_test_split(xtrain,ytrain, test_size=0.25, random_state=rd_seed)

In [213]:
# jednoduchy model stromu

depths = range(5,100)
val_acc = 0

# cross validation
for d in depths:
    dt = DecisionTreeClassifier(max_depth=d)
    dt.fit(xtr, ytr)
    score = metrics.accuracy_score(yval, dt.predict(xval))
    if val_acc < score:
        depth = d
        val_acc = score

print("optimal depth: "+str(depth))

dt = DecisionTreeClassifier(max_depth=depth)
dt.fit(xtr,ytr)
print('accuracy on train data: ' +str(metrics.accuracy_score(ytr, dt.predict(xtr))))
print('accuracy on val data: ' +str(metrics.accuracy_score(yval, dt.predict(xval))))
print('accuracy on test data: ' +str(metrics.accuracy_score(ytest, dt.predict(xtest))))

acc_dt = metrics.accuracy_score(yval, dt.predict(xval))

optimal depth: 6
accuracy on train data: 0.8861209964412812
accuracy on val data: 0.7978723404255319
accuracy on test data: 0.776


In [214]:
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier

In [215]:
# Bagging: nahodne lesy

param_grid = {
    "n_estimators": range(1,100,5),
    "max_depth": range(1,10)
}

# cross validation
param_comb = ParameterGrid(param_grid)
val_acc = 0
for params in param_comb:
    rfc = RandomForestClassifier(**params,random_state=0)
    rfc.fit(xtr, ytr)
    score = metrics.accuracy_score(yval, rfc.predict(xval))
    if val_acc < score:
        opt_params_rfc = params
        val_acc = score
print(opt_params_rfc)

# create forest
rfc = RandomForestClassifier(**opt_params_rfc)
rfc.fit(xtr, ytr)
print('accuracy on train data: ' +str(metrics.accuracy_score(ytr, rfc.predict(xtr))))
print('accuracy on val data: ' +str(metrics.accuracy_score(yval, rfc.predict(xval))))
print('accuracy on test data: ' +str(metrics.accuracy_score(ytest, rfc.predict(xtest))))

acc_rfc = metrics.accuracy_score(yval, rfc.predict(xval))

{'max_depth': 5, 'n_estimators': 6}
accuracy on train data: 0.8380782918149466
accuracy on val data: 0.7659574468085106
accuracy on test data: 0.776


In [216]:
from sklearn.ensemble import AdaBoostClassifier

In [217]:
# Boosting: AdaBoost

param_grid = {
    'n_estimators': range(1,100,5),
    'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1]
}

# cross validation
param_comb = ParameterGrid(param_grid)
val_acc = 0
for params in param_comb:
    ada = AdaBoostClassifier(**params)
    ada.fit(xtr, ytr)
    score = metrics.accuracy_score(yval, ada.predict(xval))
    if val_acc < score:
        opt_params_ada = params
        val_acc = score
print(opt_params_ada)

# create forest
ada = AdaBoostClassifier(**opt_params_ada)
ada.fit(xtr, ytr)
print('accuracy on train data: ' +str(metrics.accuracy_score(ytr, ada.predict(xtr))))
print('accuracy on val data: ' +str(metrics.accuracy_score(yval, ada.predict(xval))))
print('accuracy on test data: ' +str(metrics.accuracy_score(ytest, ada.predict(xtest))))


acc_ada = metrics.accuracy_score(yval, ada.predict(xval))

{'learning_rate': 1, 'n_estimators': 11}
accuracy on train data: 0.8202846975088968
accuracy on val data: 0.776595744680851
accuracy on test data: 0.78


In [218]:
# evaluation data to test model

test_data = pd.read_csv('evaluation.csv')

# uprav data stejnym zpusobem jako predchozi
test_data['oddil'] = test_data['cabin'].apply(edit_oddil)

n = test_data['home.dest'].str.rsplit(',',1,expand=True)[1]
test_data['home.dest'] = n[1]

n = test_data['name'].str.rsplit(',',1,expand=True)[1].str.split('.',1,expand=True)
test_data['osloveni'] = n[0]

to_drop = [item for item in test_data.columns if item not in data.columns]
test_data = test_data.drop(columns=to_drop)

test_data = encode_categories(test_data,mappers)

test_data = test_data.replace(np.nan,-1)

# vyber nejlepsi model
# print(acc_dt)
# print(acc_rfc)
# print(acc_ada)

# decission tree ma nejlepsi presnost

ypred = dt.predict(test_data)
result = pd.DataFrame(columns=['ID','Survived'])
result.ID = test_data.ID
result.Survived = ypred
result.to_csv('results.csv',index= False)